## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
#expId = 2025060100566
#expId = 2025053100267
#expId = 2025042500413
#expId = 2025061900468
#expId = 2025061900310
expId = 2025061900310
instrument='LSSTCam'
where = "exposure.id=expId AND instrument='LSSTCam'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()
#print(expRecord)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
mountErrors.azRms

In [ ]:
fig = plotMountErrors(mountData, mountErrors)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Mount_Plot_Hex_{expId}.png")

In [ ]:
from lsst.summit.utils.efdUtils import getEfdData
from astropy.time import Time, TimeDelta
hexData = getEfdData(
    client,
    "lsst.sal.MTHexapod.application",
    begin=Time("2025-06-08T00:00:00"),
    end=Time("2025-06-08T00:10:00")
)
len(hexData)

In [ ]:
azimuthData = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    begin=Time("2025-04-25T00:00:00"),
    end=Time("2025-04-25T00:10:00")
    )
len(azimuthData)

In [ ]:
import copy

In [ ]:
(mountErrors, mountData, azValues309, azModelValues309, elValues309, elModelValues309) = calculateMountErrors(expRecord, client)

In [ ]:
(mountErrors, mountData, azValues310, azModelValues310, elValues310, elModelValues310) = calculateMountErrors(expRecord, client)

In [ ]:
(mountErrors, mountData, azValues311, azModelValues311, elValues311, elModelValues311) = calculateMountErrors(expRecord, client)

In [ ]:
WARNING: TimeDeltaMissingUnitWarning: Numerical value without unit or explicit format passed to TimeDelta, assuming days [astropy.time.core]
astroquery WARNING: TimeDeltaMissingUnitWarning: Numerical value without unit or explicit format passed to TimeDelta, assuming days
Begin, End, Old 2460846.7506068675 2460846.750965093
Times, works 2460846.7506068675 2460846.750965093 <class 'astropy.time.core.Time'> <class 'astropy.time.core.Time'>

In [ ]:
Begin, End, new 2460846.7506068675 2460846.750965093
Times, fails 2460846.7506068675 2460846.750965093 <class 'astropy.time.core.Time'> <class 'astropy.time.core.Time'>
lsst.summit.utils.simonyi.mountAnalysis WARNING: Azimuth pointing model RMS error 134.214 arcsec is above threshold of 10.000 arcsec, calculating errors vs astropy.

In [ ]:
azValues[0]

In [ ]:
(2460846.750965093 - 2460846.750248642) * 86400

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
plt.plot(azValues[0:300])
plt.plot(azModelValues[0:300])

In [ ]:
type(elValues309)

In [ ]:
elValuescon = np.concatenate((elValues309, elValues310, elValues311))
elModelValuescon = np.concatenate((elModelValues309, elModelValues310, elModelValues311))
plt.plot(elValuescon)
plt.plot(elModelValuescon)


In [ ]:
azModelValues310Mod = np.zeros_like(azModelValues310)
for i in range(len(azModelValues310)):
    if azModelValues310[i] > 300.0:
        azModelValues310Mod[i] = azModelValues310[i] - 360.0
    else:
        azModelValues310Mod[i] = azModelValues310[i]
        

azValuescon = np.concatenate((azValues309, azValues310, azValues311))
azModelValuescon = np.concatenate((azModelValues309, azModelValues310Mod, azModelValues311))
plt.plot(azValuescon)
plt.plot(azModelValuescon)
plt.axhline(0.0, ls = '--', color='black')

In [ ]:
expIds = [2025061900309, 2025061900310, 2025061900311]
fig, axs = plt.subplots(1,2, figsize=(10, 8))
plt.subplots_adjust(wspace=0.5)
plt.suptitle("Mount tracking plot errors, 2024-06-19", fontsize=18)
instrument='LSSTCam'
for expId in expIds:
    where = "exposure.id=expId AND instrument='LSSTCam'"
    expRecords = butler.registry.queryDimensionRecords(
                "exposure", where=where, bind={"expId": expId}, datasets="raw"
            )
    filteredExpRecords = set(expRecords)
    expRecord = filteredExpRecords.pop()
    (mountErrors, mountData, azValTimes, azValues, azModelValues, elValTimes, elValues, elModelValues) = calculateMountErrors(expRecord, client)
    azModelValuesMod = np.zeros_like(azModelValues)
    for i in range(len(azModelValues)):
        if azModelValues[i] > 300.0:
            azModelValuesMod[i] = azModelValues[i] - 360.0
        else:
            azModelValuesMod[i] = azModelValues[i]
    axs[0].set_title("Azimuth")
    axs[0].plot(azValTimes, azValues, label=f"{expId - 2025061900000}, Meas")
    axs[0].plot(azValTimes, azModelValuesMod, ls='--', label=f"{expId - 2025061900000}, Model")
    axs[0].legend()
    axs[1].set_title("Elevation")
    axs[1].plot(elValTimes, elValues, label=f"{expId - 2025061900000}, Meas")
    axs[1].plot(elValTimes, elModelValues, ls='--', label=f"{expId - 2025061900000}, Model")
    axs[1].legend(loc='lower right')
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Model_Errors_{expIds[0]}_{expIds[2]}.png")